#### Import

In [1]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *

In [2]:
import urx
import math3d as m3d
ur = urx.Robot("192.168.1.10", use_rt=True)
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 68
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1092
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1418


<Transform:
<Orientation: 
array([[-0.00991178,  0.99283987, -0.11904095],
       [ 0.99798021,  0.00235163, -0.06348202],
       [-0.06274754, -0.11942973, -0.99085785]])>
<Vector: (-0.39682, 0.45465, 0.44730)>
>


In [75]:
from ddh import DDGripper
from scooping_primitives import HighSpeedScooping
gripper = DDGripper("ddh_scooping")
scoop = HighSpeedScooping(ur, gripper, "hss")

reading gripper config...
connecting to odrive...
found left finger
found right fingers
reading scooping config...


#### script testing

In [4]:
gripper.arm()
gripper.set_stiffness(20)
gripper.set_left_tip((158, 38))
gripper.set_right_tip((158, -38))
# gripper.set_parallel_jaw(-14.5, 0)

Setting left tip: (158, 38)
Setting right tip: (158, -38)


#### test deceleration control

In [140]:
ur.speedl([0,0,-0.4,0,0,0],0.4,10)

In [141]:
# print(ur.get_tcp_speed(wait=False)[0])
stop_dist = 0.05
z_spd = ur.get_tcp_speed(wait=False)[2]
stop_dec = (z_spd**2)/(2*stop_dist)
ur.stopl(stop_dec)
pos = ur.getl()[2]

In [142]:
stop_dist = ur.getl()[2] - pos
print(stop_dist)
print(z_spd)

-0.05322568129083427
-0.4001141268684379


In [98]:
pose = ur.get_pose()
print(pose)
print(ur.getl())

<Transform:
<Orientation: 
array([[ 0.00779014,  0.95668964, -0.29100558],
       [ 0.99974421, -0.01363056, -0.01804798],
       [-0.02123288, -0.29079055, -0.95655111]])>
<Vector: (-0.27695, 0.43693, 0.59485)>
>
[-0.2769455458607862, 0.4369331464189327, 0.5948543810792469, -2.0824007304925716, -2.0597256505179344, 0.3287233937577874]


#### scooping test

In [78]:
scoop.initialize_gripper_pose((-0.509,0.45,180))
time.sleep(0.5)
a2_init = gripper.right_a2

Setting left tip: (152.2, 31.189488223348473)
Setting right tip: (142.3, -21.663208781719646)
Setting pose: 
[-0.5134      0.45        0.2        -1.92687451 -1.92687451  0.51630447]


Robot not running: {'size': 47, 'type': 0, 'timestamp': 4987432000, 'isRobotConnected': True, 'isRealRobotEnabled': True, 'isPowerOnRobot': True, 'isEmergencyStopped': False, 'isSecurityStopped': False, 'isProgramRunning': False, 'isProgramPaused': False, 'robotMode': 4, 'controlMode': 0, 'speedFraction': 1.0, 'speedScaling': 0.0, 'speedFractionLimit': b'?'}
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/john/anaconda3/envs/scoop/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/john/anaconda3/envs/scoop/lib/python3.6/site-packages/urx/ursecmon.py", line 293, in run
    data = self._get_data()
  File "/home/john/anaconda3/envs/scoop/lib/python3.6/site-packages/urx/ursecmon.py", line 340, in _get_data
    tmp = self._s_secondary.recv(1024)
socket.timeout: timed out



In [77]:
# smack and scoop
smack_vel = 0.2 #100
smack_acc = 0.2 #0.3
stop_smack_acc = 2 #7
reverse_vel = 0.4 # 250
reverse_acc = 0.35 #1
stop_acc = 0.5 #0.5
stop_dist = 0.01

# robot accelerate
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)
# robot detect collision, then stop, close gripper, and go up
while 1:
    a2_cur = gripper.right_a2
    #print(a2_cur)
    if a2_cur - a2_init > 0.3:
        print ("Collision detected!")
        z_spd = ur.get_tcp_speed(wait=False)[2]
        stop_dec = (z_spd**2)/(2*stop_dist)
        gripper.set_stiffness(20, 'R')
        gripper.set_left_tip((158, 38))
        gripper.set_right_tip((158, -38))
        ur.stopl(stop_smack_acc)
        gripper.set_stiffness(100)
        ur.speedl([0,0,reverse_vel,0,0,0],reverse_acc,5)
        time.sleep(0.5)
        ur.stopl(stop_acc)
        break

Collision detected!
Setting left tip: (158, 38)
Setting right tip: (158, -38)


In [40]:
print(z_spd)
print(stop_dec)

-0.19452154208794392
1.8919315168135868


In [ ]:
ur.stopl()